In [15]:
!pip install wget 
!pip install scikit-learn 
!pip install "ibm-watson-machine-learning>=1.0.310" 

In [16]:
credentials = {
    "url":"https://eu-de.ml.cloud.ibm.com" ,
    "apikey":"_na8QIUga8x-gNsHoDyPRDFL8NgvPtiWSrMJkyr2em7a"
}

In [17]:
import os
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("please enter your project_id:")
project_id

'94118a53-48f5-4ade-8062-540463da4699'

In [18]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='_na8QIUga8x-gNsHoDyPRDFL8NgvPtiWSrMJkyr2em7a',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.eu-de.cloud-object-storage.appdomain.cloud')

bucket = 'medicalquestionansweringqasystem-donotdelete-pr-waqjxwjohuqinv'
object_key = 'MedQuad.csv'  # Upload this file manually

body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType(__iter__, body)

data = pd.read_csv(body)
data = data[['Question', 'Answer']].dropna().reset_index(drop=True)
data.head(10)


,Question,Answer
0,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
5,How to prevent Lymphocytic Choriomeningitis (L...,LCMV infection can be prevented by avoiding co...
6,What is (are) Parasites - Cysticercosis ?,Cysticercosis is an infection caused by the la...
7,Who is at risk for Parasites - Cysticercosis? ?,Cysticercosis is an infection caused by the la...
8,How to diagnose Parasites - Cysticercosis ?,"If you think that you may have cysticercosis, ..."
9,What are the treatments for Parasites - Cystic...,Some people with cysticercosis do not need to ...


In [19]:
from sklearn.model_selection import train_test_split

# Use only Q-A pairs
questions = data['Question']
answers = data['Answer']

q_train, q_test, a_train, a_test = train_test_split(
    questions, answers, test_size=0.3, random_state=33
)

print(f"Training samples: {len(q_train)}")
print(f"Testing samples: {len(q_test)}")


Training samples: 11484
Testing samples: 4923


In [20]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])
model_id = ModelTypes.FLAN_T5_XXL


['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01_Q', 'CODELLAMA_34B_INSTRUCT_HF', 'GRANITE_20B_MULTILINGUAL']


In [21]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.DECODING_METHOD: "Greedy",
    GenParams.RANDOM_SEED: 33,
    GenParams.REPETITION_PENALTY: 1.0,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 100
}


In [22]:
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)

model.get_details()


{'model_id': 'google/flan-t5-xxl',
 'label': 'flan-t5-xxl-11b',
 'provider': 'Google',
 'source': 'Hugging Face',
 'functions': [{'id': 'text_generation'}],
 'short_description': 'flan-t5-xxl is an 11 billion parameter model based on the Flan-T5 family.',
 'long_description': 'flan-t5-xxl (11B) is an 11 billion parameter model based on the Flan-T5 family. It is a pretrained T5 - an encoder-decoder model pre-trained on a mixture of supervised / unsupervised tasks converted into a text-to-text format, and fine-tuned on the Fine-tuned Language Net (FLAN) with instructions for better zero-shot and few-shot performance.',
 'terms_url': 'https://huggingface.co/google/flan-t5-xxl/blob/main/README.md',
 'input_tier': 'class_2',
 'output_tier': 'class_2',
 'number_params': '11b',
 'min_shot_size': 0,
 'task_ids': ['question_answering',
  'summarization',
  'retrieval_augmented_generation',
  'classification',
  'generation',
  'extraction'],
 'tasks': [{'id': 'question_answering', 'ratings': {'

In [37]:
# Example question from test set
prompt = f"Q: {q_test.iloc[0]}\nA:"
print("Question:\n", q_test.iloc[0])
params = {
    "decoding_method": "greedy"
}

response = model.generate_text(prompt=prompt, params=params)
print("🧠 Predicted Answer:\n", response)
print("✅ Ground Truth:\n", a_test.iloc[0])



Question:
 What is (are) Skin Cancer ?
🧠 Predicted Answer:
 skin cancer is a group of diseases that affect the skin .
✅ Ground Truth:
 Key Points
                    - Skin cancer is a disease in which malignant (cancer) cells form in the tissues of the skin.    - There are different types of cancer that start in the skin.     - Skin color and being exposed to sunlight can increase the risk of nonmelanoma skin cancer and actinic keratosis.     - Nonmelanoma skin cancer and actinic keratosis often appear as a change in the skin.    - Tests or procedures that examine the skin are used to detect (find) and diagnose nonmelanoma skin cancer and actinic keratosis.    - Certain factors affect prognosis (chance of recovery) and treatment options.
                
                
                    Skin cancer is a disease in which malignant (cancer) cells form in the tissues of the skin.
                    The skin is the bodys largest organ. It protects against heat, sunlight, injury, and 